In [ ]:
# %load animatePulse.py
#!/usr/bin/env python3
"""
Created on Sun Mar 20 23:52:57 2022

@author: dario
"""

from tools.io import readData, combineData, vectorizeData, \
    writePulse, readVOPfile, interpData, addLocalB0data, readPulse
    
from tools.bloch import bloch

from matplotlib import pyplot as plt
import os
from tqdm.auto import tqdm

fname = './data/dbase/GHAF-HEY2.mat'
maskName = 'brainMask'
dat = readData(fname,maskName=maskName)

dat_v = vectorizeData(dat)

[p,g] = readPulse('./UP_20deg_500us_16Tx31Rx.ini',orientation='unity')
p *= 10
nSteps = p.shape[0];
lim = bloch.getLimits()
lim['plotscale'] = 12
lim['maxRF'] = p.abs().max()

f = bloch.plotVec(p,g,dat_v,10e-6,lim)
filename  = f'img/test.png'
f.savefig(filename)

os.system('rm ./img/anim_*.png')

for i in tqdm(range(nSteps)):
    mp = p.clone();
    mg = g.clone();
    mp[i+1:,:] = 0
    mg[:,i+1:] = 0
    f = bloch.plotVec(mp,mg,dat_v,10e-6,lim,save=True)
    filename = f'img/anim_{i:03}.png'
    #f = plt.gcf()
    f.savefig(filename)
    plt.close();
#    pass

In [ ]:
os.system('pwd')
os.system(f'bash -c "convert -crop 1160x1070+00+60 +repage -delay 20 -loop 0   ./img/anim_{{000..{nSteps-2:003}}}.png ./img/animated.gif"')
os.system(f'convert -crop 1160x1070+00+60 +repage img/anim_{nSteps-1:03}.png ./img/temp.png')
os.system(f'convert ./img/animated.gif -delay 300 -loop 0 img/temp.png ./img/animated.gif')
os.system('convert ./img/animated.gif ./img/animated.mp4')
# for i in tqdm(range(nSteps)):
#     os.system(f'convert -crop 1160x0+00+60 +repage img/anim_{i:03}.png /tmp/temp.png')
#     if i==0:
#         os.system(f'convert -delay 20 -loop 0 /tmp/temp.png /tmp/animated.gif')
#     elif i==nSteps-1:
#         os.system(f'convert /tmp/animated.gif -delay 300 -loop 0 -crop 1160x0+00+60 +repage img/anim_{i:03}.png /tmp/animated.gif')
#     else:
#         os.system(f'convert /tmp/animated.gif -delay 20 -loop 0  /tmp/temp.png /tmp/animated.gif')
# os.system('mv /tmp/animated.gif ./img/animated.gif')